In [ ]:
# Install required packages
%pip install openai python-dotenv tavily-python


In [ ]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI
from tavily import TavilyClient

# Load environment variables
load_dotenv()

# Configuration
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
TAVILY_API_KEY = os.getenv('TAVILY_API_KEY')

# Initialize clients
openai_client = OpenAI(api_key=OPENAI_API_KEY) if OPENAI_API_KEY else None
tavily_client = TavilyClient(api_key=TAVILY_API_KEY) if TAVILY_API_KEY else None

print("🤖 Tavily Search Agent - Setup")
print("=" * 40)
print(f"🔑 OpenAI API: {'✅ Configured' if OPENAI_API_KEY else '❌ Missing'}")
print(f"🔍 Tavily API: {'✅ Configured' if TAVILY_API_KEY else '❌ Missing'}")

if not OPENAI_API_KEY:
    print("\n⚠️  Add OPENAI_API_KEY to your .env file")
if not TAVILY_API_KEY:
    print("\n⚠️  Add TAVILY_API_KEY to your .env file")

if OPENAI_API_KEY and TAVILY_API_KEY:
    print("\n🚀 Ready to create intelligent search agent!")


In [ ]:
class TavilySearchAgent:
    """LLM agent with Tavily search capabilities."""
    
    def __init__(self, openai_client, tavily_client):
        self.openai_client = openai_client
        self.tavily_client = tavily_client
        self.conversation_history = []
        
    def get_function_definitions(self):
        """Define Tavily search functions for OpenAI function calling."""
        return [
            {
                "type": "function",
                "function": {
                    "name": "tavily_search",
                    "description": "Search the web for comprehensive information on any topic",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {"type": "string", "description": "Search query"},
                            "max_results": {"type": "integer", "description": "Max results (1-10)", "default": 5}
                        },
                        "required": ["query"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "tavily_qna_search",
                    "description": "Get direct answers to specific questions",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {"type": "string", "description": "Question to answer"}
                        },
                        "required": ["query"]
                    }
                }
            }
        ]
    
    def execute_search(self, function_name, arguments):
        """Execute Tavily search functions."""
        try:
            if function_name == "tavily_search":
                response = self.tavily_client.search(
                    query=arguments["query"],
                    max_results=arguments.get("max_results", 5)
                )
                return {
                    "status": "success",
                    "query": arguments["query"],
                    "results": response.get("results", [])
                }
            
            elif function_name == "tavily_qna_search":
                answer = self.tavily_client.qna_search(query=arguments["query"])
                return {
                    "status": "success", 
                    "query": arguments["query"],
                    "answer": answer
                }
            
            return {"status": "error", "message": f"Unknown function: {function_name}"}
        
        except Exception as e:
            return {"status": "error", "message": str(e)}
    
    def chat(self, user_message, model="gpt-4-turbo-preview"):
        """Chat with the agent - it will search the web when needed."""
        
        self.conversation_history.append({"role": "user", "content": user_message})
        
        system_message = {
            "role": "system",
            "content": f"""You are a helpful AI assistant with real-time web search capabilities.
Current date: {datetime.now().strftime('%Y-%m-%d')}

When users ask questions requiring current information, use your search functions:
- tavily_search: For comprehensive research with multiple sources  
- tavily_qna_search: For direct factual answers

Always provide accurate, helpful responses based on the search results."""
        }
        
        messages = [system_message] + self.conversation_history
        
        try:
            # Initial OpenAI call
            response = self.openai_client.chat.completions.create(
                model=model,
                messages=messages,
                tools=self.get_function_definitions(),
                tool_choice="auto"
            )
            
            assistant_message = response.choices[0].message
            
            # Handle function calls
            if assistant_message.tool_calls:
                # Add assistant message with tool calls
                self.conversation_history.append({
                    "role": "assistant",
                    "content": assistant_message.content,
                    "tool_calls": [
                        {
                            "id": tc.id,
                            "type": "function", 
                            "function": {"name": tc.function.name, "arguments": tc.function.arguments}
                        } for tc in assistant_message.tool_calls
                    ]
                })
                
                # Execute each function call
                for tool_call in assistant_message.tool_calls:
                    function_name = tool_call.function.name
                    function_args = json.loads(tool_call.function.arguments)
                    
                    print(f"🔍 Searching: {function_args.get('query', 'N/A')}")
                    
                    # Execute search
                    result = self.execute_search(function_name, function_args)
                    
                    # Add result to conversation
                    self.conversation_history.append({
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": json.dumps(result)
                    })
                
                # Get final response
                final_response = self.openai_client.chat.completions.create(
                    model=model,
                    messages=[system_message] + self.conversation_history
                )
                
                final_message = final_response.choices[0].message.content
                self.conversation_history.append({"role": "assistant", "content": final_message})
                return final_message
            
            else:
                # No search needed
                self.conversation_history.append({"role": "assistant", "content": assistant_message.content})
                return assistant_message.content
        
        except Exception as e:
            error_msg = f"Error: {str(e)}"
            self.conversation_history.append({"role": "assistant", "content": error_msg})
            return error_msg
    
    def clear_history(self):
        """Clear conversation history."""
        self.conversation_history = []
        print("🧹 History cleared!")

# Initialize agent
if openai_client and tavily_client:
    agent = TavilySearchAgent(openai_client, tavily_client)
    print("✅ Tavily Search Agent ready!")
else:
    agent = None
    print("❌ Agent setup failed - check API keys")


In [ ]:
# Example 1: Current Events
if agent:
    print("🔍 Example 1: Current AI News")
    print("=" * 40)
    response = agent.chat("What are the latest developments in AI in 2024?")
    print(f"\n🤖 Agent: {response}")
else:
    print("❌ Agent not available - add API keys to .env file")


In [ ]:
# Example 2: Factual Question  
if agent:
    print("\n" + "="*50)
    print("❓ Example 2: Quick Fact Check")
    print("=" * 40)
    response = agent.chat("What is the current price of Bitcoin?")
    print(f"\n🤖 Agent: {response}")
else:
    print("❌ Agent not available")


In [ ]:
# Interactive Chat - Try your own questions!
def start_chat():
    """Start interactive chat with the agent."""
    if not agent:
        print("❌ Agent not available - check API keys")
        return
    
    print("\n🤖 Interactive Tavily Search Agent")
    print("=" * 50)
    print("Ask me anything! I'll search the web for current information.")
    print("Commands: 'clear' to reset, 'quit' to exit")
    print("-" * 50)
    
    while True:
        try:
            question = input("\n👤 You: ").strip()
            
            if question.lower() in ['quit', 'exit', 'q']:
                print("👋 Goodbye!")
                break
            elif question.lower() == 'clear':
                agent.clear_history()
                continue
            elif not question:
                continue
            
            print("\n🔄 Thinking...")
            response = agent.chat(question)
            print(f"\n🤖 Agent: {response}")
            
        except KeyboardInterrupt:
            print("\n👋 Chat ended!")
            break
        except Exception as e:
            print(f"\n❌ Error: {e}")

# Uncomment the line below to start interactive chat
# start_chat()

print("💡 Uncomment the last line above to start interactive chat!")
